# Streaming Agent Responses

This notebook demonstrates how to stream agent responses in real-time, allowing you to see the agent's thinking process as it happens.

## Key Concepts
- **Real-time Feedback**: See responses as they develop
- **Better UX**: Improved user experience for long-running tasks
- **Intermediate Steps**: Observe reasoning and tool calls
- **Early Termination**: Ability to stop if needed

## Stream Modes
- **"values"**: Get complete state at each step
- **"updates"**: Get only the changes/updates
- **"messages"**: Get only message updates

## What You Can Stream
- Model thinking/reasoning
- Tool calls being made
- Tool results/observations
- Final answers

## Prerequisites

Make sure you have the required packages installed:

```bash
pip install langchain langchain-community langchain-core langgraph pydantic
ollama pull qwen3
ollama serve
```

In [ ]:
# Import required modules
import time
import asyncio
from typing import Dict, Any, AsyncGenerator
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
import tools

## Basic Streaming Setup

Let's start with a basic streaming agent configuration:

In [ ]:
print("=== Basic Streaming Setup ===")

# Create model and agent for streaming
model = ChatOllama(model="qwen3")
streaming_agent = create_agent(
    model, 
    tools=[tools.web_search, tools.calculate, tools.analyze_text]
)

print("✓ Streaming agent created with multiple tools")
print("  Available tools: web_search, calculate, analyze_text")
print("  Ready for real-time streaming")

# Streaming utility functions
class StreamingUtils:
    """Utilities for handling streaming responses."""
    
    @staticmethod
    def format_timestamp() -> str:
        """Get formatted timestamp for streaming output."""
        return time.strftime("%H:%M:%S")
    
    @staticmethod
    def extract_content(message) -> str:
        """Safely extract content from message."""
        if hasattr(message, 'content') and message.content:
            return str(message.content)
        return ""
    
    @staticmethod
    def identify_message_type(message) -> str:
        """Identify the type of streaming message."""
        if hasattr(message, 'tool_calls') and message.tool_calls:
            return "tool_call"
        elif isinstance(message, ToolMessage):
            return "tool_result"
        elif isinstance(message, AIMessage):
            return "ai_response"
        elif isinstance(message, HumanMessage):
            return "human_input"
        else:
            return "unknown"
    
    @staticmethod
    def format_tool_calls(tool_calls) -> str:
        """Format tool calls for display."""
        if not tool_calls:
            return "No tools called"
        
        tool_names = []
        for call in tool_calls:
            if isinstance(call, dict):
                tool_names.append(call.get('name', 'unknown_tool'))
            else:
                tool_names.append(getattr(call, 'name', 'unknown_tool'))
        
        return f"[{', '.join(tool_names)}]"

print("\n✓ StreamingUtils helper class defined")
print("  - Timestamp formatting")
  print("  - Message content extraction")
print("  - Message type identification")
print("  - Tool call formatting")

## Simple Streaming Example

Let's demonstrate basic streaming with a multi-step query:

In [ ]:
print("=== Simple Streaming Example ===")

def simple_streaming_demo(query: str):
    """Demonstrate basic streaming functionality."""
    
    print(f"\nQuery: '{query}'")
    print(f"Started at: {StreamingUtils.format_timestamp()}")
    print("-" * 60)
    
    step_count = 0
    
    try:
        # Stream the agent's execution
        for chunk in streaming_agent.stream({
            "messages": query
        }, stream_mode="values"):
            
            step_count += 1
            timestamp = StreamingUtils.format_timestamp()
            
            # Process the chunk
            if "messages" in chunk and chunk["messages"]:
                latest_message = chunk["messages"][-1]
                message_type = StreamingUtils.identify_message_type(latest_message)
                
                print(f"[{timestamp}] Step {step_count}: {message_type.replace('_', ' ').title()}")
                
                # Handle different message types
                if message_type == "tool_call":
                    tools_called = StreamingUtils.format_tool_calls(latest_message.tool_calls)
                    print(f"  🔧 Calling tools: {tools_called}")
                
                elif message_type == "tool_result":
                    print(f"  📊 Tool completed: {getattr(latest_message, 'name', 'unknown')}")
                
                elif message_type == "ai_response":
                    content = StreamingUtils.extract_content(latest_message)
                    if content:
                        preview = content[:100] + "..." if len(content) > 100 else content
                        print(f"  💭 Agent thinking: {preview}")
            
            # Small delay to make streaming visible
            time.sleep(0.1)
        
        print("-" * 60)
        print(f"✓ Streaming completed in {step_count} steps")
        print(f"Finished at: {StreamingUtils.format_timestamp()}")
    
    except Exception as e:
        print(f"❌ Streaming error: {e}")
        return False
    
    return True

# Test with a multi-step query
test_query = "Search for the latest AI news, then calculate 25 * 4 + 100"
success = simple_streaming_demo(test_query)

if not success:
    print("\n⚠️ Streaming demo failed - falling back to regular invoke")
    try:
        result = streaming_agent.invoke({"messages": test_query})
        print(f"✓ Fallback response: {result['messages'][-1].content[:200]}...")
    except Exception as e:
        print(f"❌ Fallback also failed: {e}")

## Advanced Streaming with State Tracking

Let's create a more sophisticated streaming interface that tracks agent state:

In [ ]:
print("=== Advanced Streaming with State Tracking ===")

class AdvancedStreamingMonitor:
    """Advanced monitoring for streaming agent responses."""
    
    def __init__(self):
        self.session_stats = {
            "start_time": None,
            "total_steps": 0,
            "tool_calls": 0,
            "ai_responses": 0,
            "total_tokens": 0,
            "tools_used": set(),
            "processing_times": []
        }
        self.step_history = []
        self.last_step_time = None
    
    def start_session(self):
        """Initialize a new streaming session."""
        self.session_stats["start_time"] = time.time()
        self.last_step_time = time.time()
        print(f"🚀 Streaming session started at {StreamingUtils.format_timestamp()}")
    
    def process_chunk(self, chunk: Dict[str, Any]) -> Dict[str, Any]:
        """Process a streaming chunk and update statistics."""
        current_time = time.time()
        step_duration = current_time - self.last_step_time if self.last_step_time else 0
        
        self.session_stats["total_steps"] += 1
        self.session_stats["processing_times"].append(step_duration)
        
        step_info = {
            "step_number": self.session_stats["total_steps"],
            "timestamp": StreamingUtils.format_timestamp(),
            "duration_ms": round(step_duration * 1000, 2),
            "chunk_type": "unknown",
            "details": {}
        }
        
        # Analyze the chunk
        if "messages" in chunk and chunk["messages"]:
            latest_message = chunk["messages"][-1]
            message_type = StreamingUtils.identify_message_type(latest_message)
            step_info["chunk_type"] = message_type
            
            # Update statistics based on message type
            if message_type == "tool_call":
                self.session_stats["tool_calls"] += 1
                if hasattr(latest_message, 'tool_calls'):
                    for call in latest_message.tool_calls:
                        tool_name = call.get('name', 'unknown') if isinstance(call, dict) else getattr(call, 'name', 'unknown')
                        self.session_stats["tools_used"].add(tool_name)
                        step_info["details"]["tool_name"] = tool_name
            
            elif message_type == "ai_response":
                self.session_stats["ai_responses"] += 1
                content = StreamingUtils.extract_content(latest_message)
                step_info["details"]["content_length"] = len(content)
                self.session_stats["total_tokens"] += len(content.split()) if content else 0
        
        self.step_history.append(step_info)
        self.last_step_time = current_time
        
        return step_info
    
    def display_step(self, step_info: Dict[str, Any]):
        """Display formatted step information."""
        step_num = step_info["step_number"]
        timestamp = step_info["timestamp"]
        duration = step_info["duration_ms"]
        chunk_type = step_info["chunk_type"]
        details = step_info["details"]
        
        # Format the step display
        type_emoji = {
            "tool_call": "🔧",
            "tool_result": "📊", 
            "ai_response": "💭",
            "human_input": "👤",
            "unknown": "❓"
        }
        
        emoji = type_emoji.get(chunk_type, "❓")
        type_name = chunk_type.replace('_', ' ').title()
        
        print(f"[{timestamp}] Step {step_num}: {emoji} {type_name} ({duration}ms)")
        
        # Show relevant details
        if "tool_name" in details:
            print(f"  └─ Tool: {details['tool_name']}")
        elif "content_length" in details:
            length = details['content_length']
            print(f"  └─ Response length: {length} characters")
    
    def end_session(self) -> Dict[str, Any]:
        """End the session and return comprehensive statistics."""
        if not self.session_stats["start_time"]:
            return {"error": "Session not started"}
        
        total_duration = time.time() - self.session_stats["start_time"]
        
        summary = {
            "session_duration_seconds": round(total_duration, 2),
            "total_steps": self.session_stats["total_steps"],
            "tool_calls": self.session_stats["tool_calls"],
            "ai_responses": self.session_stats["ai_responses"],
            "estimated_tokens": self.session_stats["total_tokens"],
            "unique_tools_used": len(self.session_stats["tools_used"]),
            "tools_list": list(self.session_stats["tools_used"]),
            "average_step_time_ms": round(sum(self.session_stats["processing_times"]) * 1000 / len(self.session_stats["processing_times"]), 2) if self.session_stats["processing_times"] else 0,
            "steps_per_second": round(self.session_stats["total_steps"] / total_duration, 2) if total_duration > 0 else 0
        }
        
        return summary

print("✓ AdvancedStreamingMonitor created with comprehensive tracking:")
print("  - Session statistics and timing")
print("  - Step-by-step analysis")
print("  - Tool usage tracking")
print("  - Performance metrics")
print("  - Formatted display output")

## Testing Advanced Streaming Monitor

In [ ]:
print("=== Testing Advanced Streaming Monitor ===")

def advanced_streaming_demo(query: str, monitor: AdvancedStreamingMonitor):
    """Demonstrate advanced streaming with comprehensive monitoring."""
    
    monitor.start_session()
    print(f"Query: '{query}'")
    print("=" * 70)
    
    try:
        # Stream with advanced monitoring
        for chunk in streaming_agent.stream({
            "messages": query
        }, stream_mode="values"):
            
            # Process and display the chunk
            step_info = monitor.process_chunk(chunk)
            monitor.display_step(step_info)
            
            # Small delay for visibility
            time.sleep(0.05)
        
        # Session summary
        print("=" * 70)
        summary = monitor.end_session()
        
        print("📈 Session Summary:")
        print(f"  Total Duration: {summary['session_duration_seconds']}s")
        print(f"  Steps Processed: {summary['total_steps']}")
        print(f"  Tool Calls: {summary['tool_calls']}")
        print(f"  AI Responses: {summary['ai_responses']}")
        print(f"  Estimated Tokens: {summary['estimated_tokens']}")
        print(f"  Tools Used: {', '.join(summary['tools_list']) if summary['tools_list'] else 'None'}")
        print(f"  Avg Step Time: {summary['average_step_time_ms']}ms")
        print(f"  Processing Speed: {summary['steps_per_second']} steps/second")
        
        return True
        
    except Exception as e:
        print(f"❌ Advanced streaming error: {e}")
        return False

# Test with a complex multi-tool query
complex_query = "Search for Python machine learning tutorials, then analyze this text: 'Machine learning is transforming software development', and finally calculate the result of 15^2 + 25"

monitor = AdvancedStreamingMonitor()
success = advanced_streaming_demo(complex_query, monitor)

if not success:
    print("\n⚠️ Advanced streaming demo failed")

## Async Streaming for Better Performance

Let's implement asynchronous streaming for improved performance:

In [ ]:
print("=== Async Streaming Implementation ===")

class AsyncStreamingHandler:
    """Asynchronous streaming handler for better performance."""
    
    def __init__(self, agent):
        self.agent = agent
        self.active_streams = {}
        self.stream_counter = 0
    
    async def stream_response(self, query: str, stream_id: str = None) -> AsyncGenerator[Dict[str, Any], None]:
        """Asynchronously stream agent responses."""
        
        if not stream_id:
            stream_id = f"stream_{self.stream_counter}"
            self.stream_counter += 1
        
        self.active_streams[stream_id] = {
            "status": "active",
            "start_time": time.time(),
            "steps_processed": 0
        }
        
        try:
            # Simulate async streaming (in real implementation, use actual async agent)
            for chunk in self.agent.stream({"messages": query}, stream_mode="values"):
                
                self.active_streams[stream_id]["steps_processed"] += 1
                
                # Yield chunk with metadata
                yield {
                    "stream_id": stream_id,
                    "step": self.active_streams[stream_id]["steps_processed"],
                    "timestamp": time.time(),
                    "data": chunk
                }
                
                # Allow other coroutines to run
                await asyncio.sleep(0.01)
            
            # Mark stream as completed
            self.active_streams[stream_id]["status"] = "completed"
            self.active_streams[stream_id]["end_time"] = time.time()
            
        except Exception as e:
            self.active_streams[stream_id]["status"] = "error"
            self.active_streams[stream_id]["error"] = str(e)
            
            yield {
                "stream_id": stream_id,
                "error": str(e),
                "timestamp": time.time()
            }
    
    async def process_multiple_queries(self, queries: list) -> Dict[str, Any]:
        """Process multiple queries concurrently."""
        
        print(f"🔄 Processing {len(queries)} queries concurrently...")
        
        # Create tasks for concurrent processing
        tasks = []
        for i, query in enumerate(queries):
            stream_id = f"concurrent_stream_{i}"
            task = asyncio.create_task(self._collect_stream_results(query, stream_id))
            tasks.append((stream_id, task))
        
        # Wait for all tasks to complete
        results = {}
        for stream_id, task in tasks:
            try:
                results[stream_id] = await task
            except Exception as e:
                results[stream_id] = {"error": str(e)}
        
        return results
    
    async def _collect_stream_results(self, query: str, stream_id: str) -> Dict[str, Any]:
        """Collect all results from a stream."""
        
        results = []
        start_time = time.time()
        
        async for chunk in self.stream_response(query, stream_id):
            results.append(chunk)
        
        return {
            "query": query,
            "total_chunks": len(results),
            "duration": time.time() - start_time,
            "final_status": self.active_streams[stream_id]["status"]
        }
    
    def get_stream_status(self, stream_id: str = None) -> Dict[str, Any]:
        """Get status of active streams."""
        if stream_id:
            return self.active_streams.get(stream_id, {"error": "Stream not found"})
        
        return {
            "total_streams": len(self.active_streams),
            "active_streams": [sid for sid, info in self.active_streams.items() if info["status"] == "active"],
            "completed_streams": [sid for sid, info in self.active_streams.items() if info["status"] == "completed"],
            "error_streams": [sid for sid, info in self.active_streams.items() if info["status"] == "error"]
        }

print("✓ AsyncStreamingHandler created with capabilities:")
print("  - Asynchronous response streaming")
print("  - Concurrent query processing")
print("  - Stream status tracking")
print("  - Error handling and recovery")
print("  - Performance monitoring")

## Testing Async Streaming

In [ ]:
print("=== Testing Async Streaming ===")

async def test_async_streaming():
    """Test asynchronous streaming functionality."""
    
    handler = AsyncStreamingHandler(streaming_agent)
    
    # Test 1: Single async stream
    print("\n--- Test 1: Single Async Stream ---")
    single_query = "Calculate 10 * 5 and analyze the text 'Async streaming is powerful'"
    
    print(f"Query: {single_query}")
    print("Streaming results:")
    
    chunk_count = 0
    async for chunk in handler.stream_response(single_query):
        chunk_count += 1
        if "error" in chunk:
            print(f"  ❌ Error in chunk {chunk_count}: {chunk['error']}")
        else:
            print(f"  ✓ Chunk {chunk_count}: Step {chunk.get('step', '?')} at {StreamingUtils.format_timestamp()}")
        
        if chunk_count >= 10:  # Limit output for demo
            break
    
    print(f"Total chunks processed: {chunk_count}")
    
    # Test 2: Multiple concurrent streams
    print("\n--- Test 2: Concurrent Streams ---")
    concurrent_queries = [
        "Calculate 2 + 2",
        "Analyze text: 'Hello world'",
        "Calculate 5 * 10"
    ]
    
    start_time = time.time()
    results = await handler.process_multiple_queries(concurrent_queries)
    total_time = time.time() - start_time
    
    print(f"\n✓ Concurrent processing completed in {total_time:.2f}s")
    
    for stream_id, result in results.items():
        if "error" in result:
            print(f"  ❌ {stream_id}: Error - {result['error']}")
        else:
            print(f"  ✓ {stream_id}: {result['total_chunks']} chunks, {result['duration']:.2f}s, {result['final_status']}")
    
    # Stream status summary
    print("\n--- Stream Status Summary ---")
    status = handler.get_stream_status()
    print(f"Total streams created: {status['total_streams']}")
    print(f"Active streams: {len(status['active_streams'])}")
    print(f"Completed streams: {len(status['completed_streams'])}")
    print(f"Error streams: {len(status['error_streams'])}")

# Run the async test
try:
    await test_async_streaming()
except Exception as e:
    print(f"❌ Async streaming test failed: {e}")
    print("This might be expected in some environments")

## Streaming with User Interaction

Let's create an interactive streaming demo that allows user control:

In [ ]:
print("=== Interactive Streaming Demo ===")

class InteractiveStreamingDemo:
    """Interactive streaming demo with user controls."""
    
    def __init__(self, agent):
        self.agent = agent
        self.paused = False
        self.step_by_step = False
        self.max_steps = None
        self.current_step = 0
    
    def configure(self, step_by_step=False, max_steps=None):
        """Configure streaming behavior."""
        self.step_by_step = step_by_step
        self.max_steps = max_steps
        
        print(f"\n🔧 Streaming configured:")
        print(f"  - Step-by-step mode: {self.step_by_step}")
        print(f"  - Max steps: {self.max_steps or 'unlimited'}")
    
    def stream_with_controls(self, query: str):
        """Stream with user interaction controls."""
        
        print(f"\n🎬 Starting interactive stream for: '{query}'")
        
        if self.step_by_step:
            print("\n⏯️  Step-by-step mode enabled")
            print("   Commands: 'c' = continue, 'p' = pause, 's' = stop, 'i' = info")
        
        print("-" * 60)
        
        self.current_step = 0
        
        try:
            for chunk in self.agent.stream({"messages": query}, stream_mode="values"):
                
                self.current_step += 1
                
                # Check step limit
                if self.max_steps and self.current_step > self.max_steps:
                    print(f"\n🛑 Reached maximum steps ({self.max_steps}). Stopping stream.")
                    break
                
                # Process and display chunk
                self._display_chunk(chunk)
                
                # Handle step-by-step mode
                if self.step_by_step:
                    action = self._get_user_action()
                    if action == "stop":
                        print("\n🛑 Stream stopped by user.")
                        break
                    elif action == "pause":
                        self._handle_pause()
                
                # Small delay for readability
                time.sleep(0.1)
            
            print("-" * 60)
            print(f"✅ Stream completed. Total steps: {self.current_step}")
            
        except KeyboardInterrupt:
            print("\n\n⏹️  Stream interrupted by user (Ctrl+C)")
        except Exception as e:
            print(f"\n❌ Stream error: {e}")
    
    def _display_chunk(self, chunk):
        """Display chunk information."""
        timestamp = StreamingUtils.format_timestamp()
        
        if "messages" in chunk and chunk["messages"]:
            latest_message = chunk["messages"][-1]
            message_type = StreamingUtils.identify_message_type(latest_message)
            
            type_icons = {
                "tool_call": "🔧",
                "tool_result": "📊",
                "ai_response": "💭",
                "human_input": "👤"
            }
            
            icon = type_icons.get(message_type, "❓")
            type_name = message_type.replace('_', ' ').title()
            
            print(f"[{timestamp}] Step {self.current_step}: {icon} {type_name}")
            
            # Show additional details
            if message_type == "tool_call" and hasattr(latest_message, 'tool_calls'):
                tools = StreamingUtils.format_tool_calls(latest_message.tool_calls)
                print(f"  └─ Tools: {tools}")
            
            elif message_type == "ai_response":
                content = StreamingUtils.extract_content(latest_message)
                if content:
                    preview = content[:80] + "..." if len(content) > 80 else content
                    print(f"  └─ Response: {preview}")
    
    def _get_user_action(self):
        """Get user action in step-by-step mode (simulated)."""
        # In a real interactive environment, you would get actual user input
        # For demo purposes, we'll just continue automatically
        return "continue"
    
    def _handle_pause(self):
        """Handle pause functionality."""
        print("\n⏸️  Stream paused. Press any key to continue...")
        # In real implementation, wait for user input
        time.sleep(1)  # Simulate pause
        print("▶️  Resuming stream...\n")

# Create and test interactive demo
interactive_demo = InteractiveStreamingDemo(streaming_agent)

# Test 1: Regular streaming
print("\n--- Test 1: Regular Interactive Streaming ---")
interactive_demo.configure(step_by_step=False, max_steps=None)
interactive_demo.stream_with_controls("Calculate 7 * 8 and search for Python tutorials")

# Test 2: Step-by-step with limit
print("\n--- Test 2: Step-by-Step with Limits ---")
interactive_demo.configure(step_by_step=True, max_steps=5)
interactive_demo.stream_with_controls("Analyze this text: 'Interactive streaming provides great user control'")

## Performance Analysis and Optimization

Let's analyze streaming performance and provide optimization insights:

In [ ]:
print("=== Performance Analysis and Optimization ===")

class StreamingPerformanceAnalyzer:
    """Analyze and optimize streaming performance."""
    
    def __init__(self):
        self.benchmarks = []
        self.optimization_tips = {
            "high_latency": "Consider reducing model temperature or using a faster model",
            "many_tool_calls": "Optimize tool implementations or batch operations",
            "large_responses": "Implement response chunking or summarization",
            "slow_tools": "Cache tool results or use async tool execution",
            "memory_usage": "Implement state cleanup and garbage collection"
        }
    
    def benchmark_streaming(self, agent, queries: list, iterations: int = 3):
        """Benchmark streaming performance across multiple queries."""
        
        print(f"🏃 Running streaming benchmark...")
        print(f"  Queries: {len(queries)}")
        print(f"  Iterations per query: {iterations}")
        print("-" * 50)
        
        for query_idx, query in enumerate(queries):
            query_benchmarks = []
            
            print(f"\nQuery {query_idx + 1}: '{query[:50]}...'")
            
            for iteration in range(iterations):
                benchmark = self._benchmark_single_stream(agent, query, iteration + 1)
                query_benchmarks.append(benchmark)
                
                print(f"  Iteration {iteration + 1}: {benchmark['total_time']:.2f}s, {benchmark['total_steps']} steps")
            
            # Calculate averages for this query
            avg_benchmark = self._calculate_averages(query_benchmarks)
            avg_benchmark['query'] = query
            self.benchmarks.append(avg_benchmark)
            
            print(f"  Average: {avg_benchmark['avg_total_time']:.2f}s, {avg_benchmark['avg_total_steps']:.1f} steps")
        
        return self._generate_performance_report()
    
    def _benchmark_single_stream(self, agent, query: str, iteration: int):
        """Benchmark a single streaming session."""
        
        start_time = time.time()
        step_times = []
        tool_calls = 0
        ai_responses = 0
        total_steps = 0
        
        last_step_time = start_time
        
        try:
            for chunk in agent.stream({"messages": query}, stream_mode="values"):
                current_time = time.time()
                step_duration = current_time - last_step_time
                step_times.append(step_duration)
                
                total_steps += 1
                
                # Analyze chunk content
                if "messages" in chunk and chunk["messages"]:
                    latest_message = chunk["messages"][-1]
                    message_type = StreamingUtils.identify_message_type(latest_message)
                    
                    if message_type == "tool_call":
                        tool_calls += 1
                    elif message_type == "ai_response":
                        ai_responses += 1
                
                last_step_time = current_time
        
        except Exception as e:
            print(f"    ⚠️ Benchmark error: {e}")
        
        total_time = time.time() - start_time
        
        return {
            'iteration': iteration,
            'total_time': total_time,
            'total_steps': total_steps,
            'tool_calls': tool_calls,
            'ai_responses': ai_responses,
            'avg_step_time': sum(step_times) / len(step_times) if step_times else 0,
            'step_times': step_times
        }
    
    def _calculate_averages(self, benchmarks: list):
        """Calculate average metrics across benchmark iterations."""
        
        if not benchmarks:
            return {}
        
        return {
            'avg_total_time': sum(b['total_time'] for b in benchmarks) / len(benchmarks),
            'avg_total_steps': sum(b['total_steps'] for b in benchmarks) / len(benchmarks),
            'avg_tool_calls': sum(b['tool_calls'] for b in benchmarks) / len(benchmarks),
            'avg_ai_responses': sum(b['ai_responses'] for b in benchmarks) / len(benchmarks),
            'avg_step_time': sum(b['avg_step_time'] for b in benchmarks) / len(benchmarks),
            'iterations': len(benchmarks)
        }
    
    def _generate_performance_report(self):
        """Generate comprehensive performance report."""
        
        if not self.benchmarks:
            return {"error": "No benchmark data available"}
        
        # Overall statistics
        total_time = sum(b['avg_total_time'] for b in self.benchmarks)
        total_steps = sum(b['avg_total_steps'] for b in self.benchmarks)
        total_tool_calls = sum(b['avg_tool_calls'] for b in self.benchmarks)
        
        # Performance insights
        insights = []
        
        avg_time_per_query = total_time / len(self.benchmarks)
        if avg_time_per_query > 10:
            insights.append(self.optimization_tips["high_latency"])
        
        avg_tools_per_query = total_tool_calls / len(self.benchmarks)
        if avg_tools_per_query > 3:
            insights.append(self.optimization_tips["many_tool_calls"])
        
        return {
            "summary": {
                "total_queries": len(self.benchmarks),
                "avg_time_per_query": avg_time_per_query,
                "avg_steps_per_query": total_steps / len(self.benchmarks),
                "avg_tools_per_query": avg_tools_per_query,
                "total_benchmark_time": total_time
            },
            "optimization_insights": insights,
            "detailed_results": self.benchmarks
        }

# Run performance analysis
analyzer = StreamingPerformanceAnalyzer()

benchmark_queries = [
    "Calculate 5 + 3",
    "Analyze text: 'Performance matters'",
    "Search for AI news and calculate 10 * 2"
]

print("Running performance benchmark...")
report = analyzer.benchmark_streaming(streaming_agent, benchmark_queries, iterations=2)

print("\n" + "=" * 60)
print("📊 PERFORMANCE REPORT")
print("=" * 60)

if "error" not in report:
    summary = report["summary"]
    print(f"\n📈 Summary:")
    print(f"  Total queries tested: {summary['total_queries']}")
    print(f"  Average time per query: {summary['avg_time_per_query']:.2f}s")
    print(f"  Average steps per query: {summary['avg_steps_per_query']:.1f}")
    print(f"  Average tools per query: {summary['avg_tools_per_query']:.1f}")
    print(f"  Total benchmark time: {summary['total_benchmark_time']:.2f}s")
    
    if report["optimization_insights"]:
        print(f"\n💡 Optimization Insights:")
        for i, insight in enumerate(report["optimization_insights"], 1):
            print(f"  {i}. {insight}")
    else:
        print(f"\n✅ Performance looks good! No specific optimizations needed.")
else:
    print(f"❌ Benchmark failed: {report['error']}")

print("\n" + "=" * 60)

## Best Practices Summary

### When to Use Streaming

1. **Long-Running Tasks**
   - Complex research queries
   - Multi-step workflows
   - Document analysis
   - Data processing tasks

2. **Interactive Applications**
   - Chat interfaces
   - Real-time assistance
   - Educational tutoring
   - Code generation

3. **User Experience Priorities**
   - Immediate feedback desired
   - Progress visibility important
   - Early termination needed
   - Transparency in AI reasoning

### Stream Mode Selection

#### "values" Mode
- **Use when**: You need complete state at each step
- **Benefits**: Full context, comprehensive debugging
- **Drawbacks**: Higher bandwidth, more processing

#### "updates" Mode  
- **Use when**: You only need changes/deltas
- **Benefits**: Lower bandwidth, efficient processing
- **Drawbacks**: More complex state management

#### "messages" Mode
- **Use when**: You only care about message updates
- **Benefits**: Minimal overhead, simple processing
- **Drawbacks**: Limited context visibility

### Implementation Guidelines

1. **Error Handling**
   ```python
   try:
       for chunk in agent.stream(query):
           process_chunk(chunk)
   except Exception as e:
       handle_streaming_error(e)
   ```

2. **Performance Monitoring**
   - Track step timing
   - Monitor memory usage
   - Measure tool call frequency
   - Analyze user engagement

3. **User Controls**
   - Pause/resume functionality
   - Early termination options
   - Step-by-step mode
   - Progress indicators

### Performance Optimization

1. **Reduce Latency**
   - Use faster models for simple tasks
   - Optimize tool implementations
   - Cache frequent operations
   - Implement async processing

2. **Manage Resources**
   - Limit concurrent streams
   - Implement timeouts
   - Clean up completed streams
   - Monitor memory usage

3. **Improve User Experience**
   - Provide meaningful progress updates
   - Show estimated completion times
   - Display intermediate results
   - Allow user interruption

### Common Pitfalls

1. **Over-streaming**: Not every task benefits from streaming
2. **Poor Error Handling**: Streams can fail at any point
3. **Resource Leaks**: Clean up streams properly
4. **UI Overwhelm**: Too much information can confuse users
5. **Performance Issues**: Monitor and optimize continuously

## Use Cases for Streaming

### Research and Analysis
- **Literature Reviews**: Stream progress through multiple papers
- **Market Analysis**: Show real-time data gathering and processing
- **Code Review**: Stream through different aspects of code analysis

### Content Creation
- **Document Writing**: Show outline creation, research, writing process
- **Code Generation**: Display algorithm design, implementation, testing
- **Report Building**: Stream data collection, analysis, visualization

### Interactive Learning
- **Step-by-step Tutorials**: Show problem-solving process
- **Debugging Sessions**: Stream through troubleshooting steps
- **Concept Explanation**: Build understanding incrementally

### Business Applications
- **Data Processing**: Show progress through large datasets
- **Decision Support**: Stream through analysis factors
- **Process Automation**: Display workflow execution steps

## Conclusion

Streaming responses transform agents from black boxes into transparent, interactive systems that:

- **Engage Users**: Real-time feedback keeps users involved
- **Build Trust**: Transparency in reasoning builds confidence
- **Enable Control**: Users can guide and interrupt processes
- **Improve Experience**: Immediate feedback feels more responsive
- **Support Learning**: Seeing the process helps users understand

Streaming is particularly powerful for complex, multi-step tasks where the journey is as important as the destination. It turns AI assistance from a request-response interaction into a collaborative problem-solving experience.